In [2]:
import pandas as pd
import numpy as np
import random
import math
import timeit
import itertools
import warnings
import hdbscan
import seaborn as sns
import pickle
import feather
import gc
import matplotlib.pyplot as plt
from os.path import join, isfile
from collections import Counter, OrderedDict
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [3]:
mainPath = "../../../../zion/OpenSNP"
beacons = join(mainPath, "beacon")
ceuPath = join(beacons, "CEU")
mixedPath = join(beacons, "CEU_OpenSNP")
opensnpPath = join(beacons, "OpenSNP_reduced")
opensnpPath2 = join(beacons, "OpenSNP_constrained")

In [4]:
maf = pd.read_csv(join(ceuPath, "AF_CEU_all_chromosomes.txt"), index_col=0)

In [5]:
# CEU
beacon = pd.read_csv(join(ceuPath, "Beacon_2.txt"), index_col=0, delim_whitespace=True)

In [6]:
chr1 = maf[maf.index == "chr1"]["markerId"].values
beacon1 = beacon.loc[chr1]

In [44]:
beacon1.reset_index(inplace=True)
beacon1.to_csv(join(ceuPath, "Beacon_chr1.csv"), index=True, header=True, sep=' ')

In [7]:
insiders = beacon1.sample(20, axis=1)
insiders.to_csv(join(ceuPath, "InsidePeople2_chr1.csv"), index=True, header=True, sep=' ')
insiders = pd.read_csv(join(ceuPath, "InsidePeople2_chr1.csv"), index_col=0, delim_whitespace=True)

In [8]:
# CEU
extra = pd.read_csv(join(ceuPath, "extra.txt"), index_col=0, delim_whitespace=True)
extra1 = extra.loc[chr1]

In [9]:
outsiders = extra1.sample(20, axis=1)
outsiders.to_csv(join(ceuPath, "OutsidePeople2_chr1.csv"), index=True, header=True, sep=' ')

In [52]:
ld["Var4"].values

array(['rs12565286', 'rs6665000', 'rs6665000', ..., 'rs306901',
       'rs306901', 'rs306899'], dtype=object)

In [51]:
ld = pd.read_csv('/home/nora/data/CEU/LD/ld_new20_CEU_07_sortedM.txt')
ld

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12
0,711153,713682,CEU,rs12565286,rs11804171,1.0,1.0,4.17,7,0.033222,1.000000,0.929894
1,914761,916294,CEU,rs6665000,rs4970403,1.0,1.0,3.97,9,0.026271,1.000000,1.000000
2,914761,917172,CEU,rs6665000,rs2341362,1.0,1.0,3.97,9,0.026271,1.000000,1.000000
3,914761,918699,CEU,rs6665000,rs9777703,1.0,1.0,3.97,9,0.026271,1.000000,1.000000
4,914761,919053,CEU,rs6665000,rs9777939,1.0,1.0,3.97,9,0.026271,1.000000,1.000000
5,914761,933331,CEU,rs6665000,rs3121567,1.0,1.0,3.97,9,0.026271,1.000000,1.000000
6,914761,936897,CEU,rs6665000,rs2465126,1.0,1.0,3.97,9,0.029788,1.000000,0.878075
7,914761,938555,CEU,rs6665000,rs2341365,1.0,1.0,3.97,9,0.028092,1.000000,0.933190
8,914761,938784,CEU,rs6665000,rs15842,1.0,1.0,3.97,9,0.028092,1.000000,0.933190
9,916294,917172,CEU,rs4970403,rs2341362,1.0,1.0,3.97,9,0.026271,1.000000,1.000000


In [3]:
b= pd.read_csv('/home/nora/data/CEU/genotypes_chr4_CEU_r27_nr.b36_fwd.txt', delim_whitespace=True)
b

,rs#,alleles,chrom,pos,strand,assembly#,center,protLSID,assayLSID,panelLSID,...,NA12872,NA12873,NA12874,NA12875,NA12877,NA12878,NA12889,NA12890,NA12891,NA12892
0,rs4690249,A/T,chr4,2906,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.51540...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,TT,TT,TT,TT,NN,TT,NN,NN,TT,TT
1,rs10434467,A/C,chr4,2921,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.63679...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,CC,CC,CC,CC,NN,CC,NN,NN,CC,CC
2,rs2213695,A/C,chr4,2955,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.62572...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,AA,AA,AA,AA,NN,AA,NN,NN,AA,NN
3,rs2213698,A/C,chr4,3112,+,ncbi_b36,mcgill-gqic,urn:LSID:illumina.hapmap.org:Protocol:Golden_G...,urn:LSID:mcgill-gqic.hapmap.org:Assay:204535:1,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,AA,AA,AA,AA,NN,AA,NN,NN,AA,AA
4,rs1988508,A/T,chr4,3351,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.66089...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,AA,AA,AA,AA,NN,AA,NN,NN,AA,AA
5,rs2354196,G/T,chr4,3774,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.79189...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,TT,TT,TT,TT,NN,TT,NN,NN,TT,TT
6,rs2354194,C/T,chr4,4306,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.50911...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,CC,CC,CC,CC,NN,CC,NN,NN,CC,CC
7,rs10446886,A/G,chr4,4331,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.73404...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,GG,GG,GG,GG,NN,GG,NN,NN,GG,GG
8,rs7658348,G/T,chr4,4436,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.76205...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,GG,GG,GG,GG,NN,GG,NN,NN,GG,GG
9,rs6845730,A/T,chr4,5143,+,ncbi_b36,perlegen,urn:lsid:perlegen.hapmap.org:Protocol:Genotypi...,urn:lsid:perlegen.hapmap.org:Assay:25767.81749...,urn:lsid:dcc.hapmap.org:Panel:CEPH-30-trios:1,...,AA,AA,AA,AA,NN,AA,NN,NN,AA,AA


In [ ]:
insiders.reset_index(inplace=True)
insiders

In [ ]:
insiders.T.to_csv("InsidePeople.csv", index=True, header=True)

In [ ]:
outsiders.reset_index(inplace=True)
outsiders.T.to_csv("OutsidePeople.csv", index=True, header=True)

In [ ]:
beacon.reset_index(inplace=True)
beacon.to_csv("Beacon.csv", index=True, header=True)

### 33

In [ ]:
cc = pd.read_csv('/zion/OpenSNP/beacon/CEU/Beacon_2.txt',  delim_whitespace=True)
cc

In [ ]:
 aa = pd.read_csv('/home/nora/data/CEU/genotypes_chr4_CEU_r27_nr.b36_fwd.txt',  delim_whitespace=True)
aa = aa.drop(aa.iloc[:, 1:11].columns, axis=1)
aa

In [ ]:
aa.columns[11:]

In [ ]:
bb = pd.read_csv('/home/nora/data/CEU/genotypes_chr4_CEU_r27_nr.b36_fwd.txt',  delim_whitespace=True)

In [ ]:
bb.T